In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd #엑셀화 판다스
import time
from selenium.webdriver.common.keys import Keys #키보드
from bs4 import BeautifulSoup as bs

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd #엑셀화 판다스
import time
from selenium.webdriver.common.keys import Keys #키보드
from bs4 import BeautifulSoup as bs

# 가져온 데이터를 텍스트화 해서 저장하는 리스트들
date_site_list = []
star_list = []
content_list = []
title_list=[]
review_all = []

# a = str(input("검색어를 입력하세요: "))
# a = a.replace(" ", "+")
search = "coffee+machine"

# 검색 한 후 사이트 바로가기
driver=webdriver.Chrome(ChromeDriverManager().install())

driver.get(f"https://www.amazon.com/s?k={search}&ref=nb_sb_noss") #중간에 검색단어를 넣으면 바로 검색한 페이지로 이동하는 규칙임
driver.maximize_window()
#광고 끄기
try: search=driver.find_element_by_xpath('//*[@id="nav-main"]/div[1]/div/div/div[3]/span[1]/span/input').click()
except:
    print("지역 광고가 없습니다! 다음을 진행해 주세요.")

i=0 #상품검색 페이지 수
# while True:
while i < 3: #상품페이지 2페이지까지만! 
    i += 1
    url = driver.current_url
    try:
        # 1페이지가 완료되면 다음페이지로 넘어감
        if i != 1:
            driver.find_element_by_css_selector("""#search > div.s-desktop-width-max.s-desktop-content.s-opposite-dir.sg-row 
            > div.s-matching-dir.sg-col-16-of-20.sg-col.sg-col-8-of-12.sg-col-12-of-16 
            > div > span:nth-child(4) > div.s-main-slot.s-result-list.s-search-results.sg-row > div:nth-child(64) 
            > div.s-widget-container.s-spacing-medium.s-widget-container-height-medium.celwidget.slot=MAIN.template=PAGINATION.widgetId=pagination-button
            > div.a-section.a-text-center.s-pagination-container 
            > span.s-pagination-strip > a.s-pagination-item.s-pagination-next.s-pagination-button.s-pagination-separator""")
        else:
            pass
    except:
        print("검색한 상품의 페이지가 끝났습니다.")

    print(f"=========== 아직 {i}페이지 입니다 ===========")
    # k=0
    two = 2
    try:
        while True:

            for one in range(1,65):
                try:
                    #각 페이지에서 제품 사이트 들어가기
                    driver.find_element_by_xpath(f'//*[@id="search"]/div[1]/div[1]/div/span[3]/div[2]/div[{one}]/div/div/div/div/div[{two}]/div[1]/h2/a/span').click()
                    driver.implicitly_wait(5)
                    time.sleep(0.1)
                    soup=bs(driver.page_source, 'html.parser')

                    #상품명 가져오기
                    title_bs=soup.select('h1>span.a-size-large.product-title-word-break')
                    for P in title_bs:
                        title_list.append(P.text.strip())

                    #모든 리뷰보기로 이동하기 위한 url 가져오기
                    AllReview = soup.select("a.a-link-emphasis.a-text-bold") 
                    AllReview_url = 'https://www.amazon.com/' + AllReview[0]["href"]

                    #모든 리뷰로 이동
                    driver.get(AllReview_url) 
                    driver.implicitly_wait(5)
                    # print("넘어갔음")

                    x = 1
                    while True:
                        #모든 리뷰의 페이지 소스 가져오기
                        soup = bs(driver.page_source, "lxml") 
                        time.sleep(1)
                        try: 
                            #원하는 태그 삭제하기(번역)
                            remove = soup.select("span.a-size-base.review-text.review-text-content > span.cr-translated-review-content.aok-hidden")
                            for rm in remove:
                                rm.extract()
                                print("번역하기 소스 삭제 완료")
                        except:
                            print("번역하기를 넘어갑니다")
                            pass
                        try:
                            #[Beautiful Soup] CSS를 이용한 코드 가져오기(날짜&장소, 별점, 내용)
                            date_site = soup.select('div.a-section.celwidget > span.a-size-base.a-color-secondary.review-date') 
                            star = soup.select('div:nth-child(2) > a:nth-child(1) > i > span.a-icon-alt')
                            content = soup.select('span.a-size-base.review-text.review-text-content > span') 
                            time.sleep(1)
                            print("데이터 소스 수집 완료")
                            print(data_site, data_site[0])
                            print(star, star[0])
                            print(content, content[0])
                        except:
                            print("소스 수집 실패")

                        try:
                            #[Beautiful Soup] 가져온 코드 Text화해서 리스트에 추가하기
                            for b in range(len(date_site)):
                                date_site_list.append(date_site[b].text.replace("Reviewed in ", ""))
                                star_list.append(star[b].text.replace(" out of 5 stars", ""))
                                content_list.append(content[b].text)
                        except:
                            print("각 데이터 리스트 추가 실패")
                        
                        review_all.append(list(date_site_list))
                        review_all.append(list(star_list))
                        review_all.append(list(content_list))
                        # print(f"100 : {x+1}")

                        print("reviewall에 추가 완료")

                        # 리뷰 다음페이지로 이동하기 / 더이상 페이지 없으면 멈추기
                        try:    
                            x += 1            
                            NextPage = driver.find_element_by_xpath('//*[@id="cm_cr-pagination_bar"]/ul/li[2]/a') 
                            NextPage.click()
                            print(f"클릭 회수 : {x+1}")
                            driver.implicitly_wait(5)
                            time.sleep(0.1)
                            driver.refresh()
                            driver.implicitly_wait(5)
                            time.sleep(0.1)
                            print("새로고침 완료")
                        except:
                            print(f"{x-1}페이지에서 리뷰 수집을 끝냅니다.")
                            break
                    
                    driver.get(url)
                    # for back in range(x):
                    #     driver.back()
                    #     print(f"121back : {back}")
                    #     driver.implicitly_wait(5)
                    #     time.sleep(2)
                    #     print(f"뒤로가기 {back}")

                # 리뷰를 빠져나올 때 첫번째 검색한 페이지로 가는 부분에서 오류가 발생함. 전 페이지로 가면 리뷰페이지만 있을 수 밖에...    

                # print("98번째 지나감")
                # k = two + 1

                except:
                    print(f"one : {one}, two : {two}번째 상품이 없음.") # 해당하지 않은 제품의 개수를 확인하기 위한 출력문
                    pass
                
            two =3

    except:
        pass  
print(len(review_all[0]), len(review_all[1]), len(review_all[2]))

=========== 아직 1페이지 입니다 ===========
one : 1, two : 2번째 상품이 없음.
one : 2, two : 2번째 상품이 없음.
데이터 소스 수집 완료
소스 수집 실패
reviewall에 추가 완료
클릭 회수 : 3
새로고침 완료
데이터 소스 수집 완료
소스 수집 실패
reviewall에 추가 완료
클릭 회수 : 4
새로고침 완료
데이터 소스 수집 완료
소스 수집 실패
reviewall에 추가 완료
클릭 회수 : 5
새로고침 완료
데이터 소스 수집 완료
소스 수집 실패
reviewall에 추가 완료
클릭 회수 : 6
새로고침 완료
데이터 소스 수집 완료
소스 수집 실패
reviewall에 추가 완료
클릭 회수 : 7
새로고침 완료
데이터 소스 수집 완료
소스 수집 실패
reviewall에 추가 완료
클릭 회수 : 8
새로고침 완료
데이터 소스 수집 완료
소스 수집 실패
reviewall에 추가 완료
클릭 회수 : 9
새로고침 완료
데이터 소스 수집 완료
소스 수집 실패
reviewall에 추가 완료
클릭 회수 : 10
새로고침 완료
데이터 소스 수집 완료
소스 수집 실패
reviewall에 추가 완료
클릭 회수 : 11
새로고침 완료
데이터 소스 수집 완료
소스 수집 실패
reviewall에 추가 완료
클릭 회수 : 12
새로고침 완료
데이터 소스 수집 완료
소스 수집 실패
reviewall에 추가 완료
클릭 회수 : 13
새로고침 완료


In [2]:
print(review_all)

#print( "date_site", len(date_site),  "star", len(star), "content", len(content))

NameError: name 'review_all' is not defined

In [ ]:

driver=webdriver.Chrome(ChromeDriverManager().install())
driver.find_element_by_xpath(f'//*[@id="search"]/div[1]/div[1]/div/span[3]/div[2]/div[7]/div/div/div/div/div[2]/div[1]/h2/a/span').click()

0

In [ ]:
print(len(date_site_list), len(star_list), len(content_list))

print( "date_site", len(date_site),  "star", len(star), "content", len(content))

x = 1
for i in content:
    try:    
        print(f"================{x}번째=======================")
        print(i.text)
        x += 1
    except:
        print(f"{i}번째 오류 \n")



31 31 31
date_site 1 star 1 content 1
================1번째=======================
This machine makes perfect expresso--rich dark coffee with a tan foam.  Easy to clean the coffee part.  The milk frother is another story.  Very difficult to use correctly and also a pain to clean.  This appliance would be a 5 star but for the frother.  I use a separate tool to froth the milk/half & half.  I recommend this machine for  expresso performance- at a good price point.


In [ ]:
# 가져온 데이터를 텍스트화 해서 저장하는 리스트들
date_site_list = []
star_list = []
content_list = []
title_list=[]
review_all = []

# 사이트 가기https://www.amazon.com/ref=nav_logo
driver=webdriver.Chrome(ChromeDriverManager().install())
#data 창이 뜨고 나서 그걸 움직이는 것
#가져오고자 하는 url입력
#.get.url("URL")
driver.get("https://www.amazon.com/s?k=coffee+machine&crid=3G3G5RBAIJZCN&sprefix=%2Caps%2C211&ref=nb_sb_ss_recent_1_0_recent")
driver.maximize_window()

#광고 끄기
try: search=driver.find_element_by_xpath('//*[@id="nav-main"]/div[1]/div/div/div[3]/span[1]/span/input').click()
except:
    print("지역 광고가 없습니다! 다음을 진행해 주세요.")

soup = bs(driver.page_source, "lxml") 

# driver.close()

content = soup.select('''h2.a-size-mini.a-spacing-none.a-color-base.s-line-clamp-4 
> a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal> span.a-size-base.-color-base.a-text-normal''') 

# [Beautiful Soup] 가져온 코드 Text화해서 리스트에 추가하기
for b in content:
    print(b.text)
# NextPage = driver.find_element_by("a-size-base a-color-base a-text-normal") 
# NextPage.click()

# driver.close()

In [ ]:
len(content)

0

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd #엑셀화 판다스
import time
from selenium.webdriver.common.keys import Keys #키보드
from bs4 import BeautifulSoup as bs

# 가져온 데이터를 텍스트화 해서 저장하는 리스트들
date_site_list = []
star_list = []
content_list = []
title_list=[]
review_all = []

# a = str(input("검색어를 입력하세요: "))
# a = a.replace(" ", "+")
search = "coffee+machine"

# 검색 한 후 사이트 바로가기
driver=webdriver.Chrome(ChromeDriverManager().install())

driver.get(f"https://www.amazon.com/s?k={search}&ref=nb_sb_noss") #중간에 검색단어를 넣으면 바로 검색한 페이지로 이동하는 규칙임
driver.maximize_window()
#광고 끄기
try: search=driver.find_element_by_xpath('//*[@id="nav-main"]/div[1]/div/div/div[3]/span[1]/span/input').click()
except:
    print("지역 광고가 없습니다! 다음을 진행해 주세요.")

i=0 #상품검색 페이지 수
# while True:
while i < 3: #상품페이지 2페이지까지만! 
    i += 1
    try:
        # 1페이지가 완료되면 다음페이지로 넘어감
        if i != 1:
            driver.find_element_by_css_selector("""#search > div.s-desktop-width-max.s-desktop-content.s-opposite-dir.sg-row 
            > div.s-matching-dir.sg-col-16-of-20.sg-col.sg-col-8-of-12.sg-col-12-of-16 
            > div > span:nth-child(4) > div.s-main-slot.s-result-list.s-search-results.sg-row > div:nth-child(64) 
            > div.s-widget-container.s-spacing-medium.s-widget-container-height-medium.celwidget.slot=MAIN.template=PAGINATION.widgetId=pagination-button
            > div.a-section.a-text-center.s-pagination-container 
            > span.s-pagination-strip > a.s-pagination-item.s-pagination-next.s-pagination-button.s-pagination-separator""")
        else:
            pass
    except:
        print("검색한 상품의 페이지가 끝났습니다.")

    print(f"=========== 아직 {i}페이지 입니다 ===========")
    # k=0
    two = 2

while True:

    one = 7
    #각 페이지에서 제품 사이트 들어가기
    driver.find_element_by_xpath(f'//*[@id="search"]/div[1]/div[1]/div/span[3]/div[2]/div[{one}]/div/div/div/div/div[{two}]/div[1]/h2/a/span').click()
    driver.implicitly_wait(5)
    time.sleep(1)
    soup=bs(driver.page_source, 'html.parser')

    #상품명 가져오기
    title_bs=soup.select('h1>span.a-size-large.product-title-word-break')
    for P in title_bs:
        title_list.append(P.text.strip())

    #모든 리뷰보기로 이동하기 위한 url 가져오기
    AllReview = soup.select("a.a-link-emphasis.a-text-bold") 
    AllReview_url = 'https://www.amazon.com/' + AllReview[0]["href"]

    #모든 리뷰로 이동
    driver.get(AllReview_url) 
    driver.implicitly_wait(5)
    # print("넘어갔음")

    x = 1
    while True:
        #모든 리뷰의 페이지 소스 가져오기
        soup = bs(driver.page_source, "lxml") 

        #원하는 태그 삭제하기(번역)
        remove = soup.select("span.a-size-base.review-text.review-text-content > span.cr-translated-review-content.aok-hidden")
        for rm in remove:
            rm.extract()

        #[Beautiful Soup] CSS를 이용한 코드 가져오기(날짜&장소, 별점, 내용)
        date_site = soup.select('div.a-section.celwidget > span.a-size-base.a-color-secondary.review-date') 
        star = soup.select('div:nth-child(2) > a:nth-child(1) > i > span.a-icon-alt')
        content = soup.select('span.a-size-base.review-text.review-text-content > span') 

        #[Beautiful Soup] 가져온 코드 Text화해서 리스트에 추가하기
        for b in range(len(date_site)):
            date_site_list.append(date_site[b].text.replace("Reviewed in ", ""))
            star_list.append(star[b].text.replace(" out of 5 stars", ""))
            content_list.append(content[b].text)
    
        review_all.append(list(date_site_list))
        review_all.append(list(star_list))
        review_all.append(list(content_list))
        # print("99번 지남")

        # 리뷰 다음페이지로 이동하기 / 더이상 페이지 없으면 멈추기
        try:    
            x += 1            
            NextPage = driver.find_element_by_xpath('//*[@id="cm_cr-pagination_bar"]/ul/li[2]/a') 
            NextPage.click()
            print(f"클릭 회수 : {x+1}")
            driver.implicitly_wait(5)
            time.sleep(2)
            driver.refresh()
            driver.implicitly_wait(5)
            time.sleep(2)
        except:
            print(f"{x-1}페이지에서 리뷰 수집을 끝냅니다.")
            break
    
    
    for back in range(x):
        driver.back()
        driver.implicitly_wait(5)
        time.sleep(2)
        print(f"뒤로가기 {back}")

# 리뷰를 빠져나올 때 첫번째 검색한 페이지로 가는 부분에서 오류가 발생함. 전 페이지로 가면 리뷰페이지만 있을 수 밖에...    

# print("98번째 지나감")
# k = two + 1

print(len(review_all[0]), len(review_all[1]), len(review_all[2]))

=========== 아직 1페이지 입니다 ===========
검색한 상품의 페이지가 끝났습니다.
=========== 아직 2페이지 입니다 ===========
검색한 상품의 페이지가 끝났습니다.
=========== 아직 3페이지 입니다 ===========


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="search"]/div[1]/div[1]/div/span[3]/div[2]/div[7]/div/div/div/div/div[2]/div[1]/h2/a/span"}
  (Session info: chrome=105.0.5195.127)


In [ ]:
review_all[2]

["I bought my first espresso machine at a Starbucks in 1995 and have been making mostly cappuccino forever. I was a little reluctant to buy a machine I had never heard of, had only a few reviews from beginners and enthusiasts, and, going forward...who knows? But. Three cappuccinos in, I have just made the best one in 27 years! First impression of this machine - very impressed. I may or may not write more going forward, but I'll say a couple more things in Casabrews favor. My first day I received an email from them checking in and asking how it was going...and they sent several pdf's to help. They provide a complete second grinder mechanism in case you need to fix, adjust, clean or otherwise need a clean start. And, as of the writing of this, I got an instant $150 coupon taking the price of the machine to $500. So. Initially. Recommended! Good luck, happy brewing. Enjoy...",
 "I'm a new owner of two weeks and still learning but so far I am very happy with my purchase. The coffee & espre